In [1]:
import pandas as pd
import numpy as np
import pandas_profiling

### Reading .CSV, verifying column types and some simple statistics

In [2]:
df = pd.read_csv('./data/nba2k20-full.csv')

In [3]:
df.head()

,full_name,rating,jersey,team,position,b_day,height,weight,salary,country,draft_year,draft_round,draft_peak,college
0,LeBron James,97,#23,Los Angeles Lakers,F,12/30/84,6-9 / 2.06,250 lbs. / 113.4 kg.,$37436858,USA,2003,1,1,NaN
1,Kawhi Leonard,97,#2,Los Angeles Clippers,F,06/29/91,6-7 / 2.01,225 lbs. / 102.1 kg.,$32742000,USA,2011,1,15,San Diego State
2,Giannis Antetokounmpo,96,#34,Milwaukee Bucks,F-G,12/06/94,6-11 / 2.11,242 lbs. / 109.8 kg.,$25842697,Greece,2013,1,15,NaN
3,Kevin Durant,96,#7,Brooklyn Nets,F,09/29/88,6-10 / 2.08,230 lbs. / 104.3 kg.,$37199000,USA,2007,1,2,Texas
4,James Harden,96,#13,Houston Rockets,G,08/26/89,6-5 / 1.96,220 lbs. / 99.8 kg.,$38199000,USA,2009,1,3,Arizona State


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429 entries, 0 to 428
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   full_name    429 non-null    object
 1   rating       429 non-null    int64 
 2   jersey       429 non-null    object
 3   team         406 non-null    object
 4   position     429 non-null    object
 5   b_day        429 non-null    object
 6   height       429 non-null    object
 7   weight       429 non-null    object
 8   salary       429 non-null    object
 9   country      429 non-null    object
 10  draft_year   429 non-null    int64 
 11  draft_round  429 non-null    object
 12  draft_peak   429 non-null    object
 13  college      363 non-null    object
dtypes: int64(2), object(12)
memory usage: 47.0+ KB


### Transforming variables such as height, weight, salary, draft_round and draft_peak in numerical values instead of objects
### Transforming b_day in the year the player was born
### Fill NA values when the player has no team (Free Agent)

In [5]:
import re #importing regex lib
def transformHeight(row):
    pattern = "[0-9]+\.[0-9]*"
    x = re.findall(pattern, row)
    return x[0]

def transformWeight(row):
    pattern = "/\s*([0-9]+\.*[0-9]*)"
    x = re.findall(pattern, row)
    return x[0]

In [6]:
columns_to_transform = ['height', 'weight', 'salary', 'draft_round', 'draft_peak']
for column in columns_to_transform:
    min_before = df[column].min()
    max_before = df[column].max()
    print(f'Min {column} before transformation: {min_before}')
    print(f'Max {column} before transformation: {max_before}')
    
    if column == 'height':
        df[column] = df[column].transform(lambda row: transformHeight(row)).astype(float)
    if column == 'weight':
        df[column] = df[column].transform(lambda row: transformWeight(row)).astype(float)
    if column == 'salary':
        df[column] = df[column].str.lstrip('$').astype(float)
    if column == 'draft_round' or column == 'draft_peak':
        df.loc[df[column] == 'Undrafted', column] = -1
        df[column] = df[column].astype('int8')
        
    min_after = df[column].min()
    max_after = df[column].max()
    print(f'Min {column} after transformation: {min_after}')
    print(f'Max {column} after transformation: {max_after}')

Min height before transformation: 5-10 / 1.78
Max height before transformation: 7-4 / 2.24
Min height after transformation: 1.75
Max height after transformation: 2.24
Min weight before transformation: 170 lbs. / 77.1 kg.
Max weight before transformation: 290 lbs. / 131.5 kg.
Min weight after transformation: 77.1
Max weight after transformation: 131.5
Min salary before transformation: $100000
Max salary before transformation: $9881598
Min salary after transformation: 50000.0
Max salary after transformation: 40231758.0
Min draft_round before transformation: 1
Max draft_round before transformation: Undrafted
Min draft_round after transformation: -1
Max draft_round after transformation: 2
Min draft_peak before transformation: 1
Max draft_peak before transformation: Undrafted
Min draft_peak after transformation: -1
Max draft_peak after transformation: 60


In [7]:
df['year_born'] = pd.to_datetime(df['b_day']).dt.year

In [8]:
df.describe()

,rating,height,weight,salary,draft_year,draft_round,draft_peak,year_born
count,429.000000,429.000000,429.000000,4.290000e+02,429.000000,429.000000,429.000000,429.000000
mean,76.372960,1.995641,97.219814,8.530080e+06,2014.093240,0.932401,18.051282,1993.174825
std,5.657748,0.083514,10.536428,9.218059e+06,3.937678,0.931083,16.232458,4.115133
min,67.000000,1.750000,77.100000,5.000000e+04,2001.000000,-1.000000,-1.000000,1980.000000
25%,72.000000,1.930000,89.400000,2.000000e+06,2012.000000,1.000000,4.000000,1990.000000
50%,75.000000,2.010000,96.600000,4.380120e+06,2015.000000,1.000000,14.000000,1994.000000
75%,79.000000,2.060000,104.300000,1.220000e+07,2017.000000,1.000000,30.000000,1996.000000
max,97.000000,2.240000,131.500000,4.023176e+07,2019.000000,2.000000,60.000000,2000.000000


In [9]:
df['team'].fillna('Free Agent', inplace=True)

In [10]:
df.loc[df['position'] == 'C-F', 'position'] = 'F-C'
df.loc[df['position'] == 'F-G', 'position'] = 'G-F'

In [11]:
#profile = pandas_profiling.ProfileReport(df, title='NBA2k2020 Pandas-Profiling Report')

In [12]:
#profile

In [13]:
#profile.to_file('profile_report.html')

In [14]:
df.to_parquet('./data/dataframe.parquet')

In [16]:
df.head()

,full_name,rating,jersey,team,position,b_day,height,weight,salary,country,draft_year,draft_round,draft_peak,college,year_born
0,LeBron James,97,#23,Los Angeles Lakers,F,12/30/84,2.06,113.4,37436858.0,USA,2003,1,1,NaN,1984
1,Kawhi Leonard,97,#2,Los Angeles Clippers,F,06/29/91,2.01,102.1,32742000.0,USA,2011,1,15,San Diego State,1991
2,Giannis Antetokounmpo,96,#34,Milwaukee Bucks,G-F,12/06/94,2.11,109.8,25842697.0,Greece,2013,1,15,NaN,1994
3,Kevin Durant,96,#7,Brooklyn Nets,F,09/29/88,2.08,104.3,37199000.0,USA,2007,1,2,Texas,1988
4,James Harden,96,#13,Houston Rockets,G,08/26/89,1.96,99.8,38199000.0,USA,2009,1,3,Arizona State,1989
